In [1]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback #, CallbackList
from stable_baselines3.common.monitor import Monitor

from citylearn.data import DataSet

import pandas as pd
import json

from KBMproject import utilities as utils
from KBMproject import ATLA

In [2]:
DATASET_NAME = 'citylearn_challenge_2022_phase_2'
SAVE_DIR = 'Models/reduced features/'
LOG_DIR = 'logs/Phase3/reduced features/'
VERBOSITY = 0
DEVICE = 'cuda'
BINS = 20
TOTAL_EPISODES = 300
EVALS = 10
RESULT_DIR = '19 feature 300 episodes results/'

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
active_obs = []
for idx in schema['observations'].keys():
    value = schema['observations'][idx]['active']
    if value:
        active_obs.append(idx)
for idx in range(len(active_obs)):
    print(idx,active_obs[idx])

0 month
1 day_type
2 hour
3 outdoor_dry_bulb_temperature
4 outdoor_dry_bulb_temperature_predicted_6h
5 outdoor_dry_bulb_temperature_predicted_12h
6 outdoor_dry_bulb_temperature_predicted_24h
7 outdoor_relative_humidity
8 outdoor_relative_humidity_predicted_6h
9 outdoor_relative_humidity_predicted_12h
10 outdoor_relative_humidity_predicted_24h
11 diffuse_solar_irradiance
12 diffuse_solar_irradiance_predicted_6h
13 diffuse_solar_irradiance_predicted_12h
14 diffuse_solar_irradiance_predicted_24h
15 direct_solar_irradiance
16 direct_solar_irradiance_predicted_6h
17 direct_solar_irradiance_predicted_12h
18 direct_solar_irradiance_predicted_24h
19 carbon_intensity
20 non_shiftable_load
21 solar_generation
22 electrical_storage_soc
23 net_electricity_consumption
24 electricity_pricing
25 electricity_pricing_predicted_6h
26 electricity_pricing_predicted_12h
27 electricity_pricing_predicted_24h


In [5]:
rm_idx = [3,4,5,6,7,8,9,19,24,25,26,27]
for idx in rm_idx:
    schema['observations'][active_obs[idx]]['active'] = False

In [6]:
env = utils.make_discrete_env(schema,
                              action_bins=BINS,
                              seed=0)

eval_env = utils.make_discrete_env(schema,
                                   action_bins=BINS,
                                   seed=42)

In [7]:
env.observation_names

[['month_cos',
  'month_sin',
  'day_type_cos',
  'day_type_sin',
  'hour_cos',
  'hour_sin',
  'outdoor_relative_humidity_predicted_24h',
  'diffuse_solar_irradiance',
  'diffuse_solar_irradiance_predicted_6h',
  'diffuse_solar_irradiance_predicted_12h',
  'diffuse_solar_irradiance_predicted_24h',
  'direct_solar_irradiance',
  'direct_solar_irradiance_predicted_6h',
  'direct_solar_irradiance_predicted_12h',
  'direct_solar_irradiance_predicted_24h',
  'non_shiftable_load',
  'solar_generation',
  'electrical_storage_soc',
  'net_electricity_consumption']]

In [8]:
name = f'{env.observation_space.shape[0]} feature {TOTAL_EPISODES} episodes'

In [9]:
T = env.time_steps - 1

In [10]:
policy_kwargs = dict(net_arch=[256, 256])
agent = PPO('MlpPolicy', 
            env,
            device=DEVICE,
            policy_kwargs=policy_kwargs,
            tensorboard_log=LOG_DIR,
            verbose=VERBOSITY,
            )

ValueError: generalized State-Dependent Exploration (gSDE) can only be used with continuous actions.

In [11]:
agent.learn(total_timesteps=TOTAL_EPISODES*T,
            callback=[EvalCallback(Monitor(eval_env),
                                       eval_freq=TOTAL_EPISODES//EVALS*T,
                                       verbose=VERBOSITY),
                      ATLA.HParamCallback(),
                      ],
                tb_log_name=name,
                )

In [12]:
if SAVE_DIR is not None:
    agent.save(SAVE_DIR + name)

In [15]:
baseline_kpis, baseline_obs, baseline_a = utils.eval_agent(env,agent)
display(baseline_kpis)

cost_function
annual_peak_average                      1.000000
carbon_emissions_total                   0.903137
cost_total                               0.858539
daily_one_minus_load_factor_average      1.051263
daily_peak_average                       0.938306
electricity_consumption_total            0.904562
monthly_one_minus_load_factor_average    0.987939
ramping_average                          1.220829
zero_net_energy                          1.090996
Name: District, dtype: float64

In [18]:
kpi_savename = RESULT_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename)
    df_kpis['baseline'] = baseline_kpis
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    baseline_kpis.name = 'baseline'
    baseline_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

19 feature 300 episodes results/KPIs.csv created


In [19]:
cols = env.observation_names

In [20]:
df_sa = pd.DataFrame(baseline_obs)
df_sa.columns = cols
df_sa['actions'] = baseline_a
df_sa.to_csv(RESULT_DIR+'baseline_obs-a.csv')

In [24]:
with open(RESULT_DIR + 'schema.json', 'w') as outfile:
    json.dump(schema, outfile)